In [2]:
import pandas as pd

In [5]:
df_fs38 = pd.DataFrame(columns=['path', 'file_type', 'realm', 'frequency', 'table_id', 'project_id',
       'institution_id', 'source_id', 'experiment_id', 'member_id',
       'variable_id', 'grid_label', 'version', 'time_range'])

In [6]:
df = pd.read_csv("./file_list.txt", header=None)
df.columns = ["path"]

## DO OBS6

In [7]:
import pandas as pd
import os

# Function to split the filename into components based on '_'
def split_filename(file_path):
    # Remove the file extension and split the rest based on '_'
    filename = os.path.basename(file_path).replace('.nc', '')
    components = filename.split('_')
    # Return components in a dictionary format for DataFrame usage
    if len(components) > 6:
        return {
        'project_id': components[0],
        'source_id': components[1],
        'experiment_id': components[2],
        'version': components[3],
        'table_id': components[4],
        'variable_id': components[5],
        'time_range': components[6]
    }
    else:
        return {
        'project_id': components[0],
        'source_id': components[1],
        'experiment_id': components[2],
        'version': components[3],
        'table_id': components[4],
        'variable_id': components[5],
        'time_range': None
    }        
# Apply function to each file path and expand results into separate columns
obs_df = df[df['path'].str.contains("OBS")].copy()
obs_df[['project_id', 'source_id', 'experiment_id', 'version', 'table_id', 'variable_id', 'time_range']] = obs_df['path'].apply(lambda x: pd.Series(split_filename(x)))

## DO OBS4MIPs

In [8]:
import pandas as pd
import os

# Function to split the filename into components based on '_'
def split_filename(file_path):
    # Remove the file extension and split the rest based on '_'
    filename = os.path.basename(file_path).replace('.nc', '')
    components = filename.split('_')
    # Return components in a dictionary format for DataFrame usage
    if "-" in components[-1]:
        time_range = components[-1]
    else:
        time_range = "-".join([components[-2], components[-1]])
    return {
        'project_id': "obs4MIPs",
        'source_id': components[2],
        'version': components[4],
        'variable_id': components[0],
        'time_range': time_range} 
    
# Apply function to each file path and expand results into separate columns
obs4mips_df = df[df['path'].str.lower().str.contains("obs4mips")].copy()
obs4mips_df[['project_id', 'source_id', 'version', 'variable_id', 'time_range']] = obs4mips_df['path'].apply(lambda x: pd.Series(split_filename(x)))

In [9]:
obs4mips_df

,path,project_id,source_id,version,variable_id,time_range
2877,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,CALIOP,CALIPSO-GOCCP-v2.1,cllcalipso,20071201-20071231
2878,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,CALIOP,CALIPSO-GOCCP-v2.1,clhcalipso,20100501-20100531
2879,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,CALIOP,CALIPSO-GOCCP-v2.1,clmcalipso,20080801-20080831
2880,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,CALIOP,CALIPSO-GOCCP-v2.1,clccalipso,20080701-20080731
2881,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,CALIOP,CALIPSO-GOCCP-v2.1,cllcalipso,20090501-20090531
...,...,...,...,...,...,...
6182,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,ISCCP,V1.0,albisccp,200402-200402
6183,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,ISCCP,V1.0,albisccp,199504-199504
6186,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,ISCCP,V1.0,albisccp,200001-200001
6188,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,obs4MIPs,ISCCP,V1.0,cltisccp,198812-198812


## DO the rest

In [10]:
import pandas as pd
import os

# Function to split the filename into components based on '_'
def split_filename(file_path):
    # Remove the file extension and split the rest based on '_'
    filename = os.path.basename(file_path).replace('.nc', '')
    components = filename.split('_')
    # Return components in a dictionary format for DataFrame usage
    if "-" in components[-1]:
        time_range = components[-1]
    else:
        time_range = "-".join([components[-2], components[-1]])
    return {
        'project_id': None,
        'source_id': components[1],
        'version': components[3],
        'variable_id': components[0],
        'time_range': time_range} 
    
# Apply function to each file path and expand results into separate columns
indices_to_remove = set(obs_df.index).union(set(obs4mips_df.index))
remaining_df = df.drop(indices_to_remove)
remaining_df[['project_id', 'source_id', 'version', 'variable_id', 'time_range']] = remaining_df['path'].apply(lambda x: pd.Series(split_filename(x)))

In [11]:
df_final = pd.DataFrame(columns=df_fs38.columns)
df = pd.concat([obs_df, obs4mips_df, remaining_df])
df_final[df.columns] = df

In [12]:
df_final["frequency"] = df_final.table_id.replace({"Amon": "mon", "Omon": "mon", "OImon": "mon", "Lmon": "mon", "day": "day", "Emon": "mon", "cfMon": "mon"})

In [13]:
df_final

,path,file_type,realm,frequency,table_id,project_id,institution_id,source_id,experiment_id,member_id,variable_id,grid_label,version,time_range
0,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,mon,Amon,OBS6,NaN,NOAA-CIRES-20CR-V2,reanaly,NaN,clt,NaN,v2,187101-201212
1,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,mon,Amon,OBS6,NaN,NOAA-CIRES-20CR-V2,reanaly,NaN,hus,NaN,v2,187101-201212
2,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,mon,Amon,OBS6,NaN,NOAA-CIRES-20CR-V2,reanaly,NaN,tauu,NaN,v2,187101-201212
3,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,mon,Amon,OBS6,NaN,NOAA-CIRES-20CR-V2,reanaly,NaN,clwvi,NaN,v2,187101-201212
4,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,mon,Amon,OBS6,NaN,NOAA-CIRES-20CR-V2,reanaly,NaN,rsut,NaN,v2,187101-201212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6180,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,NaN,NaN,None,NaN,ISCCP,NaN,NaN,clisccp,NaN,V1.0,200107-200107
6184,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,NaN,NaN,None,NaN,ISCCP,NaN,NaN,clisccp,NaN,V1.0,198602-198602
6185,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,NaN,NaN,None,NaN,ISCCP,NaN,NaN,clisccp,NaN,V1.0,198808-198808
6187,/g/data/ct11/access-nri/replicas/esmvaltool/ob...,NaN,NaN,NaN,NaN,None,NaN,ISCCP,NaN,NaN,clisccp,NaN,V1.0,198407-198407


In [24]:
df_final.to_csv("access-ct11.csv")

In [25]:
import gzip

with open("access-ct11.csv", 'rb') as f_in:
    with gzip.open("access-ct11.csv.gz", 'wb') as f_out:
        f_out.writelines(f_in)


# Write the Intake ESM Catalog

In [15]:
import json

In [16]:
with open("./catalog/esm/catalog.json", 'r') as file:
    # Load the JSON data from the file into a Python object (dictionary or list)
    data = json.load(file)

In [17]:
data["id"] = 'access-nri-ct11'
data["title"] = 'access-nri-ct11'
data["description"] = "ACCESS-NRI Replica datasets for model evaluation. Includes ESMValTool data pool"
data["catalog_file"] = "access-ct11.csv.gz"

In [18]:
data['assets'] = {'column_name': 'path', 'format': 'netcdf'}

In [19]:
data['aggregation_control'] = {'variable_column_name': 'variable_id',
 'groupby_attrs': ['file_type',
  'project_id',
  'institution_id',
  'source_id',
  'experiment_id',
  'member_id',
  'frequency',
  'realm',
  'table_id',
  'variable_id',
  'grid_label',
  'version'],
 'aggregations': [{'type': 'join_existing',
   'attribute_name': 'time_range',
   'options': {'dim': 'time'}}]}

In [20]:
data['esmcat_version'] = '0.1.0'
data['catalog_file'] = 'access-ct11.csv.gz'

In [21]:
data["attributes"] = [{'column_name': 'file_type'},
 {'column_name': 'project_id'},
 {'column_name': 'institution_id'},
 {'column_name': 'source_id'},
 {'column_name': 'experiment_id'},
 {'column_name': 'member_id'},
 {'column_name': 'frequency'},
 {'column_name': 'realm'},
 {'column_name': 'table_id'},
 {'column_name': 'variable_id'},
 {'column_name': 'grid_label'},
 {'column_name': 'version'},
 {'column_name': 'time_range'}]

In [22]:
data

{'id': 'access-nri-ct11',
 'title': 'access-nri-ct11',
 'description': 'ACCESS-NRI Replica datasets for model evaluation. Includes ESMValTool data pool',
 'assets': {'column_name': 'path', 'format': 'netcdf'},
 'aggregation_control': {'variable_column_name': 'variable_id',
  'groupby_attrs': ['file_type',
   'project_id',
   'institution_id',
   'source_id',
   'experiment_id',
   'member_id',
   'frequency',
   'realm',
   'table_id',
   'variable_id',
   'grid_label',
   'version'],
  'aggregations': [{'type': 'join_existing',
    'attribute_name': 'time_range',
    'options': {'dim': 'time'}}]},
 'esmcat_version': '0.1.0',
 'catalog_file': 'access-ct11.csv.gz',
 'attributes': [{'column_name': 'file_type'},
  {'column_name': 'project_id'},
  {'column_name': 'institution_id'},
  {'column_name': 'source_id'},
  {'column_name': 'experiment_id'},
  {'column_name': 'member_id'},
  {'column_name': 'frequency'},
  {'column_name': 'realm'},
  {'column_name': 'table_id'},
  {'column_name': 'v

In [23]:
with open('catalog.json', 'w') as json_file:
    json.dump(data, json_file)
